# Government Disability Benefits Analysis

**Author:** Luke Steuber  
**Date:** February 2026

This notebook analyzes three major government disability benefits datasets:

1. **SSA SSDI/SSI Annual Data** (1960-2024) - Social Security disability program statistics
2. **SSA State Agency Workload** (2001-2021) - Disability determination processing and approval rates
3. **VA Disability Compensation** (2019-2026) - Veterans Affairs disability claims and compensation

## Key Findings

- **7.3 million SSDI recipients** receiving Social Security Disability Insurance
- **4.9 million SSI recipients** receiving Supplemental Security Income
- **$173+ billion annually** in total disability benefits
- **~33% initial approval rate** for disability claims
- **5.7+ million veterans** receiving VA disability compensation

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

%matplotlib inline

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## 1. Load and Explore SSA SSDI/SSI Data

This dataset contains historical beneficiary counts from 1960-2024 for both SSDI (Social Security Disability Insurance) and SSI (Supplemental Security Income) programs.

In [ ]:
# Load SSA SSDI/SSI data
with open('../ssa_ssdi_ssi_annual.json', 'r') as f:
    ssa_data = json.load(f)

print("Dataset Structure:")
print(f"Top-level keys: {list(ssa_data.keys())}")
print(f"\nYears covered: {ssa_data['metadata']['years_covered']}")
print(f"Fetch date: {ssa_data['metadata']['fetch_date']}")

print("\n=== Key Statistics (November 2024) ===")
key_stats = ssa_data['key_statistics']
print(f"Total receiving disability benefits: {key_stats['total_people_receiving_disability_benefits']['total']:,}")
print(f"  - Social Security only: {key_stats['total_people_receiving_disability_benefits']['social_security_only']:,}")
print(f"  - SSI only: {key_stats['total_people_receiving_disability_benefits']['ssi_only']:,}")
print(f"  - Both programs: {key_stats['total_people_receiving_disability_benefits']['both_programs']:,}")

print("\nSSDI (Disability Insurance):")
print(f"  Total beneficiaries: {key_stats['ssdi']['total_di_beneficiaries']:,}")
print(f"  Disabled workers: {key_stats['ssdi']['disabled_workers']:,}")
print(f"  Total monthly benefits: ${key_stats['ssdi']['total_monthly_benefits_millions']:,}M")
print(f"  Avg monthly benefit (workers): ${key_stats['ssdi']['avg_monthly_benefit_disabled_workers']:,.2f}")

print("\nSSI (Supplemental Security Income):")
print(f"  Total recipients: {key_stats['ssi']['total_recipients']:,}")
print(f"  Ages 18-64: {key_stats['ssi']['aged_18_64']:,}")
print(f"  Total monthly payments: ${key_stats['ssi']['total_payments_millions']:,}M")
print(f"  Avg monthly payment: ${key_stats['ssi']['avg_monthly_payment']:,.2f}")

## 2. SSDI Historical Trends (1960-2024)

Extract and visualize the growth in SSDI disabled worker beneficiaries over time.

In [ ]:
# Extract SSDI historical data
ssdi_years = []
ssdi_disabled_workers = []

for year, data in ssa_data['ssdi']['beneficiaries_by_year'].items():
    year_int = int(year)
    # Look for disabled workers count in various possible fields
    if 'disabled_workers_and_dependents' in data:
        count = data['disabled_workers_and_dependents']
    elif 'disabled_workers' in data:
        count = data['disabled_workers']
    else:
        continue
    
    if count is not None:
        ssdi_years.append(year_int)
        ssdi_disabled_workers.append(count)

# Create DataFrame
ssdi_df = pd.DataFrame({
    'year': ssdi_years,
    'disabled_workers': ssdi_disabled_workers
}).sort_values('year')

print(f"SSDI data points: {len(ssdi_df)}")
print(f"Year range: {ssdi_df['year'].min()} - {ssdi_df['year'].max()}")
print(f"\nFirst 5 years:")
print(ssdi_df.head())
print(f"\nLast 5 years:")
print(ssdi_df.tail())

In [ ]:
# Plot SSDI growth over time
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(ssdi_df['year'], ssdi_df['disabled_workers'] / 1_000_000, 
        linewidth=2.5, marker='o', markersize=4, color='#2E86AB')

ax.set_xlabel('Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Disabled Workers (Millions)', fontsize=12, fontweight='bold')
ax.set_title('SSDI Disabled Worker Beneficiaries (1960-2024)', 
             fontsize=14, fontweight='bold', pad=20)

ax.grid(True, alpha=0.3)
ax.set_xlim(1960, 2025)

# Add annotations for key milestones
current_value = ssdi_df[ssdi_df['year'] == ssdi_df['year'].max()]['disabled_workers'].values[0]
ax.annotate(f'{current_value/1_000_000:.1f}M beneficiaries\nin {ssdi_df["year"].max()}',
            xy=(ssdi_df['year'].max(), current_value/1_000_000),
            xytext=(ssdi_df['year'].max() - 10, current_value/1_000_000 + 0.5),
            fontsize=10, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
            arrowprops=dict(arrowstyle='->', lw=1.5))

plt.tight_layout()
plt.show()

# Calculate growth rate
start_val = ssdi_df.iloc[0]['disabled_workers']
end_val = ssdi_df.iloc[-1]['disabled_workers']
growth_pct = ((end_val - start_val) / start_val) * 100
print(f"\nGrowth: {start_val:,.0f} ({ssdi_df.iloc[0]['year']}) → {end_val:,.0f} ({ssdi_df.iloc[-1]['year']})")
print(f"Total increase: {growth_pct:.1f}%")

## 3. SSDI vs SSI Comparison

Compare current beneficiary counts and average monthly benefits between the two programs.

In [ ]:
# Prepare comparison data
program_data = pd.DataFrame({
    'Program': ['SSDI', 'SSI'],
    'Total Beneficiaries': [
        key_stats['ssdi']['total_di_beneficiaries'],
        key_stats['ssi']['total_recipients']
    ],
    'Avg Monthly Benefit': [
        key_stats['ssdi']['avg_monthly_benefit_disabled_workers'],
        key_stats['ssi']['avg_monthly_payment']
    ]
}
)

print("Program Comparison:")
print(program_data.to_string(index=False))

# Calculate total annual spending (approximate)
ssdi_annual = key_stats['ssdi']['total_monthly_benefits_millions'] * 12 / 1000  # Billions
ssi_annual = key_stats['ssi']['total_payments_millions'] * 12 / 1000  # Billions
total_annual = ssdi_annual + ssi_annual

print(f"\nEstimated Annual Spending:")
print(f"  SSDI: ${ssdi_annual:.1f}B")
print(f"  SSI: ${ssi_annual:.1f}B")
print(f"  Total: ${total_annual:.1f}B")

In [ ]:
# Create comparison charts
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Beneficiaries comparison
axes[0].bar(program_data['Program'], 
            program_data['Total Beneficiaries'] / 1_000_000,
            color=['#2E86AB', '#A23B72'], alpha=0.8, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Beneficiaries (Millions)', fontsize=11, fontweight='bold')
axes[0].set_title('Total Beneficiaries by Program\n(November 2024)', 
                  fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(program_data['Total Beneficiaries']):
    axes[0].text(i, v/1_000_000 + 0.2, f'{v/1_000_000:.1f}M', 
                 ha='center', fontsize=11, fontweight='bold')

# Average benefit comparison
axes[1].bar(program_data['Program'], 
            program_data['Avg Monthly Benefit'],
            color=['#2E86AB', '#A23B72'], alpha=0.8, edgecolor='black', linewidth=1.5)
axes[1].set_ylabel('Average Monthly Benefit ($)', fontsize=11, fontweight='bold')
axes[1].set_title('Average Monthly Benefit\n(November 2024)', 
                  fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(program_data['Avg Monthly Benefit']):
    axes[1].text(i, v + 50, f'${v:,.0f}', 
                 ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## 4. SSA State Agency Workload - Approval Rates

Analyze disability determination workload data including approval rates over time.

In [ ]:
# Load SSA state agency workload data
with open('../ssa_state_agency_workload.json', 'r') as f:
    workload_data = json.load(f)

print("SSA State Agency Workload Dataset:")
print(f"Data coverage: {workload_data['metadata']['coverage']['fiscal_year_workload']['years']}")
print(f"Total records: {workload_data['metadata']['total_records']:,}")
print(f"\nKey definitions:")
for term, definition in workload_data['metadata']['key_terms'].items():
    print(f"  {term}: {definition}")

In [ ]:
# Extract national trends data
national_trends = pd.DataFrame(workload_data['analysis']['national_trends'])

print(f"National trends data points: {len(national_trends)}")
print(f"\nColumns: {list(national_trends.columns)}")
print(f"\nFirst few years:")
print(national_trends[['fiscal_year', 'total_determinations', 'total_favorable', 
                        'total_allowance_rate_pct', 'adult_allowance_rate_pct']].head())

In [ ]:
# Plot approval rates over time
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(national_trends['fiscal_year'], national_trends['total_allowance_rate_pct'], 
        linewidth=2.5, marker='o', markersize=5, label='Overall Allowance Rate', color='#E63946')
ax.plot(national_trends['fiscal_year'], national_trends['adult_allowance_rate_pct'], 
        linewidth=2.5, marker='s', markersize=5, label='Adult Allowance Rate', color='#457B9D')
ax.plot(national_trends['fiscal_year'], national_trends['child_allowance_rate_pct'], 
        linewidth=2.5, marker='^', markersize=5, label='Child Allowance Rate', color='#2A9D8F')

ax.set_xlabel('Fiscal Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Allowance Rate (%)', fontsize=12, fontweight='bold')
ax.set_title('Disability Determination Allowance Rates (2001-2021)\nPercentage of Claims Approved', 
             fontsize=14, fontweight='bold', pad=20)

ax.grid(True, alpha=0.3)
ax.legend(loc='best', fontsize=10, framealpha=0.9)
ax.set_ylim(25, 50)

# Add horizontal line at 33%
ax.axhline(y=33, color='red', linestyle='--', alpha=0.5, linewidth=1.5, label='~33% benchmark')

plt.tight_layout()
plt.show()

# Show summary statistics
print("\nAllowance Rate Summary (2001-2021):")
print(f"Overall: {national_trends['total_allowance_rate_pct'].mean():.1f}% average")
print(f"Adult: {national_trends['adult_allowance_rate_pct'].mean():.1f}% average")
print(f"Child: {national_trends['child_allowance_rate_pct'].mean():.1f}% average")

## 5. VA Disability Compensation by State

Analyze Veterans Affairs disability compensation data showing the top states by number of veterans receiving benefits.

In [ ]:
# Load VA disability compensation data
with open('../va_disability_compensation.json', 'r') as f:
    va_data = json.load(f)

print("VA Disability Compensation Dataset:")
print(f"Data currency: {va_data['metadata']['data_currency']['claims_data']}")
print(f"\nTotal veterans receiving VA disability: {va_data['veterans_by_rating']['total_veterans_receiving_compensation']:,}")
print(f"Fiscal year: {va_data['veterans_by_rating']['fiscal_year']}")

In [ ]:
# Extract veterans by state data
state_data = []
for state_code, data in va_data['veterans_by_state'].items():
    if isinstance(data, dict) and 'state_name' in data:
        state_data.append({
            'state_code': state_code,
            'state_name': data['state_name'],
            'total_veterans': data.get('total_living_veterans_2023', 0),
            'pension_recipients_2019': data.get('pension_recipients_2019', 0)
        })

state_df = pd.DataFrame(state_data)
state_df = state_df.sort_values('total_veterans', ascending=False)

print(f"States with data: {len(state_df)}")
print(f"\nTop 10 states by veteran population:")
print(state_df[['state_name', 'total_veterans', 'pension_recipients_2019']].head(10).to_string(index=False))

In [ ]:
# Plot top 20 states by veteran population
top_20_states = state_df.head(20).copy()

fig, ax = plt.subplots(figsize=(12, 10))

bars = ax.barh(range(len(top_20_states)), 
               top_20_states['total_veterans'] / 1000,
               color='#264653', alpha=0.8, edgecolor='black', linewidth=1)

ax.set_yticks(range(len(top_20_states)))
ax.set_yticklabels(top_20_states['state_name'])
ax.set_xlabel('Total Living Veterans (Thousands)', fontsize=12, fontweight='bold')
ax.set_title('Top 20 States by Veteran Population\n(VetPop2023 Model)', 
             fontsize=14, fontweight='bold', pad=20)

ax.grid(axis='x', alpha=0.3)
ax.invert_yaxis()

# Add value labels
for i, (idx, row) in enumerate(top_20_states.iterrows()):
    ax.text(row['total_veterans']/1000 + 20, i, 
            f"{row['total_veterans']/1000:,.0f}K",
            va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nTop 5 states account for {top_20_states.head(5)['total_veterans'].sum() / state_df['total_veterans'].sum() * 100:.1f}% of all veterans")

## 6. VA Disability Rating Distribution

Examine how veterans are distributed across different disability rating percentages.

In [ ]:
# Extract veterans by rating percentage
rating_data = []
for rating, data in va_data['veterans_by_rating']['by_rating_percentage'].items():
    rating_data.append({
        'rating': rating.replace('_percent', '%'),
        'veterans': data['veterans'],
        'pct_of_total': data['pct_of_total'],
        'description': data.get('description', '')
    })

rating_df = pd.DataFrame(rating_data)
print("Veterans by Disability Rating:")
print(rating_df.to_string(index=False))

In [ ]:
# Plot rating distribution
fig, ax = plt.subplots(figsize=(14, 7))

colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(rating_df)))

bars = ax.bar(range(len(rating_df)), 
              rating_df['veterans'] / 1000,
              color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

ax.set_xticks(range(len(rating_df)))
ax.set_xticklabels(rating_df['rating'], rotation=45, ha='right')
ax.set_ylabel('Veterans (Thousands)', fontsize=12, fontweight='bold')
ax.set_xlabel('Disability Rating', fontsize=12, fontweight='bold')
ax.set_title('VA Disability Compensation Recipients by Rating Percentage\n(FY 2024 - 5.7M Total)', 
             fontsize=14, fontweight='bold', pad=20)

ax.grid(axis='y', alpha=0.3)

# Add percentage labels on top of bars
for i, (idx, row) in enumerate(rating_df.iterrows()):
    ax.text(i, row['veterans']/1000 + 20, 
            f"{row['pct_of_total']:.1f}%",
            ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

## Summary

This analysis of government disability benefits reveals:

### SSDI/SSI Programs (SSA)
- **11.3 million people** receive disability benefits through Social Security programs
- **7.3 million** receive SSDI (average $1,542/month)
- **4.9 million** receive SSI (average $698/month)  
- **$173+ billion** in combined annual benefits
- SSDI beneficiaries grew from 455K in 1960 to 7.2M+ in 2024

### Disability Determination Process
- **~33% initial approval rate** on average (2001-2021)
- Adult approval rates slightly lower than child rates
- Significant variation by state and fiscal year

### VA Disability Compensation
- **5.7+ million veterans** receive disability compensation (FY 2024)
- California, Texas, and Florida have the largest veteran populations
- 70-100% disability ratings account for the largest share of recipients
- PACT Act (2022) expanded eligibility for toxic exposure conditions

These three datasets together illustrate the scale of government disability support programs serving over 17 million Americans.